In [1]:
import sympy as sp
import numpy as np
from model import S, R, T, to_line, tan_simplify, Maclaurin_series, simplify_array, Taylor_series, truncate, diff_on_S1, jacobian
import model as m
from sympy import sqrt, acos, atan, symbols
from numpy.linalg import norm, inv
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook
plt.ion()

In [111]:
r , constraints = m.get_leg_position()
Z = sp.symbols('z_a, z_d, z_y')
M = sp.symbols(r'\alpha_h, \alpha_r, \alpha_b')
z_a, z_d, z_y = Z
Beta = m.beta_A, m.beta_D, m.beta_Y
U = m.u_h, m.u_r, m.u_b
a_h, a_r, a_b = M
subs = []

for z, u in zip(M, U):
    subs.extend([(sp.sin(u), (z - 1/z)/2j),  (sp.cos(u), (z + 1/z)/2)])
    
for z, beta in zip(Z,Beta):
    subs.extend([(sp.sin(beta), (z - 1/z)/2j),  (sp.cos(beta), (z + 1/z)/2)])
        
r = [sp.simplify(r_i.subs(subs)) for r_i in r]
constraints = [sp.simplify(sp.expand_trig(c).subs(subs)) for c in constraints]
#r[0] = sp.expand(r[0]* z_a * z_d * z_y)
#r[1] = sp.expand(r[1]*z_a*z_d*z_d*z_y)
#r[2]= sp.expand(r[2]*z_y*z_d)
constraints[0] = truncate(sp.expand(constraints[0]))
constraints[1] = sp.expand(constraints[1]*a_h*z_d)
constraints[2] = sp.expand(constraints[2]*z_y*a_r)

#
# want d/dt 
# 
F, w = jacobian(r)

In [127]:
G, w_ref = jacobian(constraints)
dG_z = G[0:3, ::2]
dG_a = G[0:3, 1::2]


### Goal
In the body frame, we have the $i$ th leg 
$$ (x,y,z)_i^T = f_i(\theta^i_t, \theta^i_m, \theta^i_b)$$
we want to choose a function $\theta^i_t, \theta^i_m, \theta^i_b = \gamma(s)$ such that $s \in [0, 1]$

Each leg consists of two 4R linkages and a parallelogram.

Two key pivot points are given by the horizontal plane of the D brace.

The two RRRR linkages define two derived angles.

In [ ]:
beta_D, beta_Y = sp.symbols(r'\beta_D, \beta_y', real=True)
u_r, u_h = sp.symbols('u_r, u_h', real=True)

In [ ]:
Leg_Pivot = R(beta_D) @ T(m.D_section_e, 0) @ R(-beta_D)
Leg_Bottom = Leg_Pivot @ R(beta_Y) @ T(m.leg_pivot_to_foot_x, m.leg_pivot_to_foot_y)

In [ ]:
# For R(beta_Y)
Y_inner = R(beta_Y) @ T(m.Y_i,m.Y_h) @ R(-beta_Y)
Y_outer = R(beta_Y) @ T(m.Y_o,m.Y_h) @ R(-beta_Y)
D_bottom = R(beta_D) @ T(0, m.D_section_b) @ R(-beta_D) 
Top_Motor = T(m.motor_offset_x, m.motor_offset_top)   
Top_Motor_Anchor = Top_Motor @ R(u_r) @ T(0, m.motor_arm_top) @ R(-u_r) 
Bottom_Motor = T(m.motor_offset_x, m.motor_offset_bottom)
Bottom_Motor_Anchor = Bottom_Motor @ R(u_h) @ T(0, m.motor_arm_bottom) @ R(-u_h)

In [ ]:
## Bottom Linkage constraint
C_b = ((Bottom_Motor_Anchor - D_bottom).T) @ (Bottom_Motor_Anchor - D_bottom)
simplify_array(C_b)
f_b = sp.simplify(sp.expand(C_b[2,2] - m.l_b**2))
# f_b = f_b.subs(u_h, sp.asin(u_h))
F_b = sp.lambdify((u_h, beta_D),f_b )
theta = np.linspace(-np.pi/2, np.pi/2, 30)
interval = np.linspace(-1/2, 1/2, 30)
xz_i, yz_i = np.meshgrid(interval, theta)
plt.contour(xz_i, yz_i, F_b(xz_i,yz_i), [0])

In [ ]:
eq = sp.collect(sp.expand_trig(f_b), beta_D)
f_beta_d = tan_simplify(eq, beta_D, implicit=False, positive_branch=True)
beta_d_series = (Maclaurin_series(f_beta_d, u_h, order=1) - sp.pi)
f_beta_d_series = sp.lambdify(u_h, beta_d_series)
plt.contour(xz_i,yz_i, F_b(xz_i,yz_i), [0])
plt.plot(theta, f_beta_d_series(theta), ':')

In [ ]:
## Top Linkage Constraint
C_t = ((Top_Motor_Anchor - Y_inner).T )@  (Top_Motor_Anchor - Y_inner)
C_t = simplify_array(C_t)
        
f_t = sp.simplify(sp.expand(C_t[2,2] - m.l_d**2))
# f_t = sp.simplify(f_t.subs(u_r, sp.atan(u_r)))
F_t = sp.lambdify((u_r, beta_Y), f_t)

eq = sp.collect(sp.expand_trig(f_t), beta_Y)
f_beta_Y_i = tan_simplify(eq, beta_Y, implicit=True)
f_beta_Y = tan_simplify(eq, beta_Y, implicit=False)
beta_Y_series = Taylor_series(f_beta_Y, u_r, 0, order=1)
f_beta_Y_series = sp.lambdify(u_r, beta_Y_series)
xz, yz = np.meshgrid(theta, theta)
plt.contour(xz,yz, F_t(xz,yz), [0])
#plt.contour(xz,yz, sp.lambdify((u_r, beta_Y), sp.re(f_beta_Y_i))(xz,yz), [0], linestyles='dotted')
plt.plot(theta, f_beta_Y_series(theta), ':')
beta_Y_series

In [ ]:
subs = [(beta_Y, beta_Y_series), (beta_D, beta_d_series)] 

In [ ]:

p = Leg_Pivot @ m.o
mt = Top_Motor @ m.o
mb = Bottom_Motor @ m.o

#plt.plot(*to_line(Leg_Pivot @ m.o, m.o))
#plt.plot(*to_line(Leg_Bottom @ m.o, p))
#plt.plot(*to_line(Y_inner @ m.o, m.o))
#plt.plot(*to_line(Y_outer @ m.o, m.o))
#plt.plot(*to_line(Y_outer @ p, p))
#plt.plot(*to_line(D_bottom@ m.o, p))
#plt.plot(*to_line(D_bottom@ m.o, m.o))
#plt.plot(*to_line(Y_outer @ p, Y_outer @ m.o))
#plt.plot(*to_line(Y_outer @ p,Leg_Bottom @ m.o ))
#plt.plot(*to_line(Top_Motor @ m.o , m.o))
#plt.plot(*to_line(Bottom_Motor@m.o, m.o))
#plt.plot(*to_line(Bottom_Motor_Anchor@m.o, mb))
#plt.plot(*to_line(Top_Motor_Anchor @m.o, mt))

In [ ]:
foot = simplify_array(Leg_Bottom) @m.o
foot[0] = tan_simplify(foot[0], beta_Y)
foot[1] = tan_simplify(foot[1], beta_Y)


In [ ]:
foot_x = sp.lambdify((u_r, u_h), foot[0].subs(subs))
foot_y = sp.lambdify((u_r, u_h), foot[1].subs(subs))

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(xz,yz, foot_x(xz,yz) - foot_x(0,0))

ax.set_xlabel('Top Motor Angle')
ax.set_ylabel('Bottom Motor Angle')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(xz,yz, foot_y(xz,yz) - foot_y(0,0))
a = ax.plot([0],[0], [0])

In [ ]:
plt.figure()
plt.contour(xz,yz, foot_y(xz,yz) - foot_y(0,0), [0, 2, 5, 10, 15])

In [ ]:
# Level Set 
# h =  foot_y(b_d, b_y) - foot_y(0, 0)
# which is h = −32.0sin(𝛽𝐷)−76.4852927038918cos(𝛽𝑦−0.197395559849881)+68.710755215083
beta_d_ref = -sp.asin(2.39016539699662*sp.cos(beta_Y - 0.197395559849881))
f_beta_ref = sp.lambdify(beta_Y, beta_d_ref) 


In [ ]:
foot[1].diff(beta_D), foot[1].diff(beta_Y)

In [ ]:
beta_d_series - beta_d_ref

In [ ]:
print(beta_ref)

In [ ]:
sp.expand_trig(f_b).subs(sp.cos(beta_D), beta_D).subs(sp.sin(beta_D), sp.sqrt(1 - beta_D**2))

In [ ]:
sp.solveset(f_b.subs(u_h, 0))

In [ ]:
from scipy.optimize import root

In [ ]:
f_b_lambda = lambda X: [abs(sp.lambdify((beta_D, u_h), f_b)(X[0], X[1])), X[1]]
root(f_b_lambda, x0=[0, 0]).x[0]

In [ ]:
_

In [ ]:
f_t_lambda =  lambda X: [abs(sp.lambdify((beta_Y, u_r), f_t)(X[0], X[1])), X[1]]
root(f_t_lambda, x0=[0, 0]).x[0]

In [ ]:
f_b

In [ ]:
a[0].__dict__